# Image Folder (2)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)

In [4]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers = 2)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(3,6,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
        nn.Conv2d(6,16,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
        nn.Linear(16*13*29, 120),
        nn.ReLU(),
        nn.Linear(120,2),
        )
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [6]:
#testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [7]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [8]:
total_batch = len(data_loader)

epochs = 30
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Completed!')

[Epoch:1] cost = 0.5152819156646729
[Epoch:2] cost = 0.475181519985199
[Epoch:3] cost = 0.432015597820282
[Epoch:4] cost = 0.4061194658279419
[Epoch:5] cost = 0.41239863634109497
[Epoch:6] cost = 0.4114011526107788
[Epoch:7] cost = 0.3594701588153839
[Epoch:8] cost = 0.24038594961166382
[Epoch:9] cost = 0.2373272180557251
[Epoch:10] cost = 0.15535128116607666
[Epoch:11] cost = 0.13329049944877625
[Epoch:12] cost = 0.13190452754497528
[Epoch:13] cost = 0.11252997815608978
[Epoch:14] cost = 0.0884430930018425
[Epoch:15] cost = 0.10193900763988495
[Epoch:16] cost = 0.07880398631095886
[Epoch:17] cost = 0.07376877218484879
[Epoch:18] cost = 0.056300535798072815
[Epoch:19] cost = 0.04965246841311455
[Epoch:20] cost = 0.0559622123837471
[Epoch:21] cost = 0.05894168093800545
[Epoch:22] cost = 0.049556292593479156
[Epoch:23] cost = 0.0533159077167511
[Epoch:24] cost = 0.041594140231609344
[Epoch:25] cost = 0.05871745944023132
[Epoch:26] cost = 0.09752975404262543
[Epoch:27] cost = 0.0963801667

In [9]:
torch.save(net.state_dict(), "./model/model.pth")

In [10]:
new_net = CNN().to(device)

In [11]:
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [12]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0828,  0.0160, -0.0037, -0.0133,  0.0986], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([-0.0828,  0.0160, -0.0037, -0.0133,  0.0986], device='cuda:0',
       grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

In [13]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform = trans)

In [14]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [15]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy : ', accuracy.item())

Accuracy :  0.9130434989929199
